# Word Vectors - Sentiment Analysis

This notebook is a continuation of my previous notebook based on the kaggle challenge: Bag of Words Meets Bag of Popcorn, in which the goal of the challenge is to correctly determine the sentiment of an author's review based on the text information.

In order to perform machine learning, the data being fed into the algorithm needs to be numerical, in my previous notebook, we represented words numerically using the Bag of Words. Here, we will use a more advanced technique that aims to capture the meaning of the word called word vectors.

# What are word vectors?

In the simplest of terms, a word vector is a numerical representation of a word. Each word has a unique meaning and a word vector aims to capture this unique meaning by representing it as a vector of numbers. To create the word vectors we will use Word2Vec, which is a neural network published by Google.

Word2Vec is a unsupervised learning algorithm, meaning it does not require labels to learn the meaning of the words. The network aims to cluster together words with similar meaning (i.e. cat, dog, kitten, puppy).

In [30]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "./Data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "./Data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "./Data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Train Review Size: {}".format(train['review'].size))
print("Test Review Size: {}".format(test['review'].size))
print("Unlabled Train Review Size: {}".format(unlabeled_train['review'].size))

Train Review Size: 25000
Test Review Size: 25000
Unlabled Train Review Size: 50000


In [6]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)



In [8]:
# Load the punkt tokenizer
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [10]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

In [12]:
print (len(sentences))

795538


In [13]:
print(sentences[0])
print(sentences[1])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [14]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-03-11 17:57:47,123 : INFO : 'pattern' package not found; tag filters are not available for English
2019-03-11 17:57:47,130 : INFO : collecting all words and their counts
2019-03-11 17:57:47,131 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-11 17:57:47,212 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-03-11 17:57:47,278 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2019-03-11 17:57:47,350 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2019-03-11 17:57:47,444 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-03-11 17:57:47,523 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-03-11 17:57:47,597 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-03-11 17:57:47,668 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-03-11 17:57:47,741 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-03-11 17:57:47,816 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-03-11 17:57:47,890 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-03-11 17:57:47,981 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

2019-03-11 17:57:54,268 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2019-03-11 17:57:54,382 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2019-03-11 17:57:54,490 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2019-03-11 17:57:54,577 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2019-03-11 17:57:54,661 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2019-03-11 17:57:54,715 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2019-03-11 17:57:54,716 : INFO : Loading a fresh vocabulary
2019-03-11 17:57:54,799 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2019-03-11 17:57:54,800 : INFO : min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2019-03-11 17:5

2019-03-11 17:58:45,199 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-11 17:58:45,200 : INFO : EPOCH - 3 : training on 17798082 raw words (12750672 effective words) took 16.4s, 779202 effective words/s
2019-03-11 17:58:46,216 : INFO : EPOCH 4 - PROGRESS: at 6.23% examples, 792185 words/s, in_qsize 7, out_qsize 0
2019-03-11 17:58:47,220 : INFO : EPOCH 4 - PROGRESS: at 12.63% examples, 798525 words/s, in_qsize 7, out_qsize 0
2019-03-11 17:58:48,224 : INFO : EPOCH 4 - PROGRESS: at 18.98% examples, 798218 words/s, in_qsize 7, out_qsize 0
2019-03-11 17:58:49,236 : INFO : EPOCH 4 - PROGRESS: at 25.16% examples, 792949 words/s, in_qsize 8, out_qsize 0
2019-03-11 17:58:50,239 : INFO : EPOCH 4 - PROGRESS: at 31.50% examples, 793952 words/s, in_qsize 6, out_qsize 1
2019-03-11 17:58:51,240 : INFO : EPOCH 4 - PROGRESS: at 37.84% examples, 796041 words/s, in_qsize 8, out_qsize 0
2019-03-11 17:58:52,246 : INFO : EPOCH 4 - PROGRESS: at 44.32% examples, 800167 words/s, in_

In [28]:
model.wv.doesnt_match("cat dog lizard toilet".split())

'toilet'

In [27]:
model.wv.most_similar("man")

[('woman', 0.6196308135986328),
 ('lady', 0.6027864217758179),
 ('lad', 0.5857887268066406),
 ('farmer', 0.5506047010421753),
 ('monk', 0.5343555212020874),
 ('millionaire', 0.5326489210128784),
 ('men', 0.5268594026565552),
 ('guy', 0.5151544213294983),
 ('soldier', 0.513985276222229),
 ('boy', 0.5056132078170776)]

In [29]:
model.wv.vectors.shape

(16490, 300)

So after we trained our word2vec neural network on our unlabled dataset, we have a model which contains the word vectors. The word vectors are essentially a 2D numpy array. The rows represent the number of words in the model's vocabulary and the number of columns represent the size of the feature vectors.

## Vector Averaging

One challenge with the IMDB dataset is that not all reviews are of the same length. Therefore, in order to use our traditional machine learning algorithms, we need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

Each word vector contains 300 numbers, one way to ensure that all features are the same length is to simply take the average of all the word vectors in any given review. This will return another 300 dimensional vector. 

**Note** we remove stop words for this operation since they will just add noise.

In [49]:
def makeFeatureVec(words, model, num_features):
    #Pre initialize a numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    
    nwords = 0.
    
    #Index2Word is a function that returns a list of the model's vocab, convert it to a set (for speed)
    index2word_set = set(model.wv.index2word)
    
    #Loop over each word in the review, if it is in model's vocab than add it into featureVec
    for word in words:
        if word in index2word_set:
            nwords += 1.
            featureVec = np.add(featureVec,model.wv[word])
    
    #Divide the featureVec by the total number of words added to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    #Initialize a counter
    counter = 0
    
    #Pre Initialize a 2D numpy array (for speed)
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype='float32')
    
    #Loop over reviews and create an average featureVec for each one.
    #Then, add it into reviewFeatureVecs
    for review in reviews:
        if counter%1000 == 0:
            print("Review {} of {}".format(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        
        counter = counter + 1
    return reviewFeatureVecs

In [50]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
print("Preprocessing the reviews")
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
print("Finished Preprocessing")

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, 300 )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, 300 )
print("Finished Creating average feature vecs")

Preprocessing the reviews
Finished Preprocessing
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review

In [55]:
print(trainDataVecs.shape)
print(testDataVecs.shape)

(25000, 300)
(25000, 300)


Now that we have our average vector for all reviews, we can train our random forest classifier on this data and submit a kaggle competition with the test data.

In [56]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting the random forest model")
forest.fit(trainDataVecs,train['sentiment'])
print("Fitting Complete")

Fitting the random forest model
Fitting Complete


In [57]:
# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

With word vectors and vector averaging, we were able to achieve a score of 0.82868 on kaggle.

## Vector Quantization

Another technique besides vector averaging is called vector quantization, which is basically a technique in which you group similar vectors together. We can perform this grouping by using a clustering algorithm like K-means.

In K-means we choose the hyperparameter K. This selection is mostly just trail and error, and according to the kaggle tutorial, the best results they got was K = 5.

In [83]:
from sklearn.cluster import KMeans
import time

print("Starting K Means Algorithm")
start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.vectors
num_clusters = word_vectors.shape[0] // 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("K Means Algorithm Completed:\nTime Durations = {}".format(elapsed))

Starting K Means Algorithm
K Means Algorithm Completed:
Time Durations = 374.3183710575104


In [84]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [85]:
for cluster in range(0,10):
    print("\nCluster {}: ".format(cluster + 1))
    
    #Find all the words with that cluster number and print them out
    words = []
    for i in word_centroid_map:
        if(word_centroid_map[i] == cluster):
            words.append(i)
    print(words)


Cluster 1: 
['ultra', 'unnecessarily', 'excessively', 'overkill', 'overuse']

Cluster 2: 
['kolchak']

Cluster 3: 
['operations']

Cluster 4: 
['naschy', 'fran', 'alain', 'molina', 'maestro', 'fenech']

Cluster 5: 
['bums']

Cluster 6: 
['progressive']

Cluster 7: 
['texture', 'sepia']

Cluster 8: 
['fired', 'stoned', 'smashed', 'busted', 'punched', 'pinned', 'bashed']

Cluster 9: 
['stamp', 'surly']

Cluster 10: 
['fenn']


In [86]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [87]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [89]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


So from using the 'BagOfCentroids', scored 0.84348 on the kaggle challenge. Which is a little bit better than the vector averaging technique.

In [91]:
result

array([1, 0, 1, ..., 0, 1, 1])